In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt


In [2]:
def load_model():
    print("Loading tokenizer and model...")
    tokenizer = AutoTokenizer.from_pretrained("shailja/fine-tuned-codegen-2B-Verilog")
    model = AutoModelForCausalLM.from_pretrained("shailja/fine-tuned-codegen-2B-Verilog")
    return tokenizer, model

In [9]:
def calculate_perplexity(text, tokenizer, model):
    # Truncate text if it's too long for the model
    max_length = 512  # Set a reasonable value
    input_ids = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length).input_ids
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        loss = outputs.loss
        perplexity = torch.exp(loss)
    return perplexity.item()

In [4]:
# Analyze two ALU Verilog files (clean and potentially trojaned)
def analyze_alu_files(clean_file_path, suspicious_file_path):
    tokenizer, model = load_model()

    # Read clean ALU code
    print(f"Analyzing clean ALU file: {clean_file_path}")
    with open(clean_file_path, 'r', encoding='utf-8') as f:
        clean_code = f.read()

    clean_perplexity = calculate_perplexity(clean_code, tokenizer, model)
    print(f"Clean ALU perplexity: {clean_perplexity:.4f}")

    # Read suspicious ALU code
    print(f"\nAnalyzing suspicious ALU file: {suspicious_file_path}")
    with open(suspicious_file_path, 'r', encoding='utf-8') as f:
        suspicious_code = f.read()

    suspicious_perplexity = calculate_perplexity(suspicious_code, tokenizer, model)
    print(f"Suspicious ALU perplexity: {suspicious_perplexity:.4f}")

    # Calculate difference and percentage
    difference = suspicious_perplexity - clean_perplexity
    percent_increase = (difference / clean_perplexity) * 100

    print(f"\nPerplexity difference: {difference:.4f}")
    print(f"Percent increase: {percent_increase:.2f}%")

    # Simple detection threshold (adjust as needed)
    threshold_percent = 20
    if percent_increase > threshold_percent:
        print(f"\n⚠️ WARNING: Potential hardware trojan detected!")
        print(f"Perplexity increase exceeds {threshold_percent}%")
    else:
        print(f"\n✓ No significant anomalies detected (threshold: {threshold_percent}%)")

    # Plot the results
    plot_comparison(clean_perplexity, suspicious_perplexity)

    return clean_perplexity, suspicious_perplexity

In [5]:
def plot_comparison(clean_perplexity, suspicious_perplexity):
    plt.figure(figsize=(8, 5))

    bars = plt.bar(['Clean ALU', 'Suspicious ALU'],
                   [clean_perplexity, suspicious_perplexity],
                   color=['green', 'red'], alpha=0.7)

    plt.ylabel('Perplexity (lower is more expected)')
    plt.title('Perplexity Comparison: Clean vs. Suspicious ALU Code')
    plt.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.1,
                 f'{height:.4f}', ha='center', va='bottom')

    plt.savefig('alu_perplexity_comparison.png')
    plt.close()
    print("Saved perplexity comparison plot to 'alu_perplexity_comparison.png'")


In [10]:
if __name__ == "__main__":
    # Set these to your actual file paths
    clean_alu_path = "alu_clean.v"
    suspicious_alu_path = "alu_suspicious.v"

    analyze_alu_files(clean_alu_path, suspicious_alu_path)

Loading tokenizer and model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at shailja/fine-tuned-codegen-2B-Verilog were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.20.attn.causal_mask', 'transformer.h.21.attn.causal_mask', 'transformer.h.22.attn.causal_mask', 'transformer.h.23.attn.causal_mask', 'transformer.h.24.attn.causal_mask', 'transformer.h.25.attn.causal_mask', 'transformer.h.26.attn.causal_mask', 'transformer.h.27.attn.causal_mask', 'transformer.h.28.attn.causal_mask', 'transformer.h.29.attn.causal_mask', 'transformer.h.3.attn

Analyzing clean ALU file: alu_clean.v
Clean ALU perplexity: 2.0105

Analyzing suspicious ALU file: alu_suspicious.v
Suspicious ALU perplexity: 3.2570

Perplexity difference: 1.2465
Percent increase: 62.00%

⚠️ WARNING: Potential hardware trojan detected!
Perplexity increase exceeds 20%
Saved perplexity comparison plot to 'alu_perplexity_comparison.png'
